From https://open-meteo.com/

https://open-meteo.com/en/docs/historical-weather-api?start_date=2020-01-01&location_mode=csv_coordinates&csv_coordinates=37.019356,+-7.930440&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,weather_code,surface_pressure,cloud_cover,pressure_msl,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,boundary_layer_height,wet_bulb_temperature_2m,total_column_integrated_water_vapour,is_day,sunshine_duration,albedo,snow_depth_water_equivalent,shortwave_radiation,diffuse_radiation,global_tilted_irradiance,shortwave_radiation_instant,diffuse_radiation_instant,global_tilted_irradiance_instant,direct_radiation,direct_normal_irradiance,terrestrial_radiation,direct_radiation_instant,direct_normal_irradiance_instant,terrestrial_radiation_instant,temperature_2m_spread,precipitation_spread,dew_point_2m_spread,snowfall_spread,shortwave_radiation_spread,direct_radiation_spread,pressure_msl_spread,cloud_cover_low_spread,cloud_cover_mid_spread,cloud_cover_high_spread,wind_speed_10m_spread,wind_speed_100m_spread,wind_direction_10m_spread,wind_direction_100m_spread,wind_gusts_10m_spread,soil_temperature_0_to_7cm_spread,soil_temperature_7_to_28cm_spread,soil_temperature_28_to_100cm_spread,soil_temperature_100_to_255cm_spread,soil_moisture_0_to_7cm_spread,soil_moisture_7_to_28cm_spread,soil_moisture_28_to_100cm_spread,soil_moisture_100_to_255cm_spread&tilt=40&daily=weather_code,temperature_2m_mean,apparent_temperature_mean,daylight_duration,rain_sum,wind_speed_10m_max,shortwave_radiation_sum,temperature_2m_max,apparent_temperature_max,sunrise,sunshine_duration,snowfall_sum,wind_gusts_10m_max,et0_fao_evapotranspiration,temperature_2m_min,apparent_temperature_min,sunset,precipitation_sum,precipitation_hours,wind_direction_10m_dominant

In [2]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [10]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 37.019356,
    "longitude": -7.93044,
    "start_date": "2020-01-01",
    "end_date": "2025-03-20",
    "daily": ["weather_code", "temperature_2m_mean", "apparent_temperature_mean", "daylight_duration", "rain_sum", "wind_speed_10m_max", "shortwave_radiation_sum", "temperature_2m_max", "apparent_temperature_max", "sunrise", "sunshine_duration", "snowfall_sum", "wind_gusts_10m_max", "et0_fao_evapotranspiration", "temperature_2m_min", "apparent_temperature_min", "sunset", "precipitation_sum", "precipitation_hours", "wind_direction_10m_dominant"],
    "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", "rain", "snowfall", "snow_depth", "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm", "soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm", "soil_moisture_0_to_7cm", "soil_moisture_7_to_28cm", "soil_moisture_28_to_100cm", "soil_moisture_100_to_255cm", "weather_code", "surface_pressure", "cloud_cover", "pressure_msl", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_100m", "wind_direction_10m", "wind_direction_100m", "wind_gusts_10m", "boundary_layer_height", "wet_bulb_temperature_2m", "total_column_integrated_water_vapour", "is_day", "sunshine_duration", "albedo", "snow_depth_water_equivalent", "shortwave_radiation", "diffuse_radiation", "global_tilted_irradiance", "shortwave_radiation_instant", "diffuse_radiation_instant", "global_tilted_irradiance_instant", "direct_radiation", "direct_normal_irradiance", "terrestrial_radiation", "direct_radiation_instant", "direct_normal_irradiance_instant", "terrestrial_radiation_instant", "temperature_2m_spread", "precipitation_spread", "dew_point_2m_spread", "snowfall_spread", "shortwave_radiation_spread", "direct_radiation_spread", "pressure_msl_spread", "cloud_cover_low_spread", "cloud_cover_mid_spread", "cloud_cover_high_spread", "wind_speed_10m_spread", "wind_speed_100m_spread", "wind_direction_10m_spread", "wind_direction_100m_spread", "wind_gusts_10m_spread", "soil_temperature_0_to_7cm_spread", "soil_temperature_7_to_28cm_spread", "soil_temperature_28_to_100cm_spread", "soil_temperature_100_to_255cm_spread", "soil_moisture_0_to_7cm_spread", "soil_moisture_7_to_28cm_spread", "soil_moisture_28_to_100cm_spread", "soil_moisture_100_to_255cm_spread"],
    "tilt": 40
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
hourly_rain = hourly.Variables(5).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(8).ValuesAsNumpy()
hourly_soil_temperature_7_to_28cm = hourly.Variables(9).ValuesAsNumpy()
hourly_soil_temperature_28_to_100cm = hourly.Variables(10).ValuesAsNumpy()
hourly_soil_temperature_100_to_255cm = hourly.Variables(11).ValuesAsNumpy()
hourly_soil_moisture_0_to_7cm = hourly.Variables(12).ValuesAsNumpy()
hourly_soil_moisture_7_to_28cm = hourly.Variables(13).ValuesAsNumpy()
hourly_soil_moisture_28_to_100cm = hourly.Variables(14).ValuesAsNumpy()
hourly_soil_moisture_100_to_255cm = hourly.Variables(15).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(16).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(17).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(18).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(19).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(20).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(21).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(22).ValuesAsNumpy()
hourly_et0_fao_evapotranspiration = hourly.Variables(23).ValuesAsNumpy()
hourly_vapour_pressure_deficit = hourly.Variables(24).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(25).ValuesAsNumpy()
hourly_wind_speed_100m = hourly.Variables(26).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(27).ValuesAsNumpy()
hourly_wind_direction_100m = hourly.Variables(28).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(29).ValuesAsNumpy()
hourly_boundary_layer_height = hourly.Variables(30).ValuesAsNumpy()
hourly_wet_bulb_temperature_2m = hourly.Variables(31).ValuesAsNumpy()
hourly_total_column_integrated_water_vapour = hourly.Variables(32).ValuesAsNumpy()
hourly_is_day = hourly.Variables(33).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(34).ValuesAsNumpy()
hourly_albedo = hourly.Variables(35).ValuesAsNumpy()
hourly_snow_depth_water_equivalent = hourly.Variables(36).ValuesAsNumpy()
hourly_shortwave_radiation = hourly.Variables(37).ValuesAsNumpy()
hourly_diffuse_radiation = hourly.Variables(38).ValuesAsNumpy()
hourly_global_tilted_irradiance = hourly.Variables(39).ValuesAsNumpy()
hourly_shortwave_radiation_instant = hourly.Variables(40).ValuesAsNumpy()
hourly_diffuse_radiation_instant = hourly.Variables(41).ValuesAsNumpy()
hourly_global_tilted_irradiance_instant = hourly.Variables(42).ValuesAsNumpy()
hourly_direct_radiation = hourly.Variables(43).ValuesAsNumpy()
hourly_direct_normal_irradiance = hourly.Variables(44).ValuesAsNumpy()
hourly_terrestrial_radiation = hourly.Variables(45).ValuesAsNumpy()
hourly_direct_radiation_instant = hourly.Variables(46).ValuesAsNumpy()
hourly_direct_normal_irradiance_instant = hourly.Variables(47).ValuesAsNumpy()
hourly_terrestrial_radiation_instant = hourly.Variables(48).ValuesAsNumpy()
hourly_temperature_2m_spread = hourly.Variables(49).ValuesAsNumpy()
hourly_precipitation_spread = hourly.Variables(50).ValuesAsNumpy()
hourly_dew_point_2m_spread = hourly.Variables(51).ValuesAsNumpy()
hourly_snowfall_spread = hourly.Variables(52).ValuesAsNumpy()
hourly_shortwave_radiation_spread = hourly.Variables(53).ValuesAsNumpy()
hourly_direct_radiation_spread = hourly.Variables(54).ValuesAsNumpy()
hourly_pressure_msl_spread = hourly.Variables(55).ValuesAsNumpy()
hourly_cloud_cover_low_spread = hourly.Variables(56).ValuesAsNumpy()
hourly_cloud_cover_mid_spread = hourly.Variables(57).ValuesAsNumpy()
hourly_cloud_cover_high_spread = hourly.Variables(58).ValuesAsNumpy()
hourly_wind_speed_10m_spread = hourly.Variables(59).ValuesAsNumpy()
hourly_wind_speed_100m_spread = hourly.Variables(60).ValuesAsNumpy()
hourly_wind_direction_10m_spread = hourly.Variables(61).ValuesAsNumpy()
hourly_wind_direction_100m_spread = hourly.Variables(62).ValuesAsNumpy()
hourly_wind_gusts_10m_spread = hourly.Variables(63).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm_spread = hourly.Variables(64).ValuesAsNumpy()
hourly_soil_temperature_7_to_28cm_spread = hourly.Variables(65).ValuesAsNumpy()
hourly_soil_temperature_28_to_100cm_spread = hourly.Variables(66).ValuesAsNumpy()
hourly_soil_temperature_100_to_255cm_spread = hourly.Variables(67).ValuesAsNumpy()
hourly_soil_moisture_0_to_7cm_spread = hourly.Variables(68).ValuesAsNumpy()
hourly_soil_moisture_7_to_28cm_spread = hourly.Variables(69).ValuesAsNumpy()
hourly_soil_moisture_28_to_100cm_spread = hourly.Variables(70).ValuesAsNumpy()
hourly_soil_moisture_100_to_255cm_spread = hourly.Variables(71).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
    start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = hourly.Interval()),
    inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["soil_temperature_7_to_28cm"] = hourly_soil_temperature_7_to_28cm
hourly_data["soil_temperature_28_to_100cm"] = hourly_soil_temperature_28_to_100cm
hourly_data["soil_temperature_100_to_255cm"] = hourly_soil_temperature_100_to_255cm
hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
hourly_data["soil_moisture_7_to_28cm"] = hourly_soil_moisture_7_to_28cm
hourly_data["soil_moisture_28_to_100cm"] = hourly_soil_moisture_28_to_100cm
hourly_data["soil_moisture_100_to_255cm"] = hourly_soil_moisture_100_to_255cm
hourly_data["weather_code"] = hourly_weather_code
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["boundary_layer_height"] = hourly_boundary_layer_height
hourly_data["wet_bulb_temperature_2m"] = hourly_wet_bulb_temperature_2m
hourly_data["total_column_integrated_water_vapour"] = hourly_total_column_integrated_water_vapour
hourly_data["is_day"] = hourly_is_day
hourly_data["sunshine_duration"] = hourly_sunshine_duration
hourly_data["albedo"] = hourly_albedo
hourly_data["snow_depth_water_equivalent"] = hourly_snow_depth_water_equivalent
hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
hourly_data["diffuse_radiation"] = hourly_diffuse_radiation
hourly_data["global_tilted_irradiance"] = hourly_global_tilted_irradiance
hourly_data["shortwave_radiation_instant"] = hourly_shortwave_radiation_instant
hourly_data["diffuse_radiation_instant"] = hourly_diffuse_radiation_instant
hourly_data["global_tilted_irradiance_instant"] = hourly_global_tilted_irradiance_instant
hourly_data["direct_radiation"] = hourly_direct_radiation
hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance
hourly_data["terrestrial_radiation"] = hourly_terrestrial_radiation
hourly_data["direct_radiation_instant"] = hourly_direct_radiation_instant
hourly_data["direct_normal_irradiance_instant"] = hourly_direct_normal_irradiance_instant
hourly_data["terrestrial_radiation_instant"] = hourly_terrestrial_radiation_instant
hourly_data["temperature_2m_spread"] = hourly_temperature_2m_spread
hourly_data["precipitation_spread"] = hourly_precipitation_spread
hourly_data["dew_point_2m_spread"] = hourly_dew_point_2m_spread
hourly_data["snowfall_spread"] = hourly_snowfall_spread
hourly_data["shortwave_radiation_spread"] = hourly_shortwave_radiation_spread
hourly_data["direct_radiation_spread"] = hourly_direct_radiation_spread
hourly_data["pressure_msl_spread"] = hourly_pressure_msl_spread
hourly_data["cloud_cover_low_spread"] = hourly_cloud_cover_low_spread
hourly_data["cloud_cover_mid_spread"] = hourly_cloud_cover_mid_spread
hourly_data["cloud_cover_high_spread"] = hourly_cloud_cover_high_spread
hourly_data["wind_speed_10m_spread"] = hourly_wind_speed_10m_spread
hourly_data["wind_speed_100m_spread"] = hourly_wind_speed_100m_spread
hourly_data["wind_direction_10m_spread"] = hourly_wind_direction_10m_spread
hourly_data["wind_direction_100m_spread"] = hourly_wind_direction_100m_spread
hourly_data["wind_gusts_10m_spread"] = hourly_wind_gusts_10m_spread
hourly_data["soil_temperature_0_to_7cm_spread"] = hourly_soil_temperature_0_to_7cm_spread
hourly_data["soil_temperature_7_to_28cm_spread"] = hourly_soil_temperature_7_to_28cm_spread
hourly_data["soil_temperature_28_to_100cm_spread"] = hourly_soil_temperature_28_to_100cm_spread
hourly_data["soil_temperature_100_to_255cm_spread"] = hourly_soil_temperature_100_to_255cm_spread
hourly_data["soil_moisture_0_to_7cm_spread"] = hourly_soil_moisture_0_to_7cm_spread
hourly_data["soil_moisture_7_to_28cm_spread"] = hourly_soil_moisture_7_to_28cm_spread
hourly_data["soil_moisture_28_to_100cm_spread"] = hourly_soil_moisture_28_to_100cm_spread
hourly_data["soil_moisture_100_to_255cm_spread"] = hourly_soil_moisture_100_to_255cm_spread

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe.to_csv("hourly_weather_data.csv.zip")
hourly_dataframe.describe()

Coordinates 37.01230239868164°N -7.95513916015625°E
Elevation 22.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s


,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,...,wind_direction_100m_spread,wind_gusts_10m_spread,soil_temperature_0_to_7cm_spread,soil_temperature_7_to_28cm_spread,soil_temperature_28_to_100cm_spread,soil_temperature_100_to_255cm_spread,soil_moisture_0_to_7cm_spread,soil_moisture_7_to_28cm_spread,soil_moisture_28_to_100cm_spread,soil_moisture_100_to_255cm_spread
count,45744.000000,45744.000000,45744.000000,45744.000000,45744.000000,45744.000000,45744.000000,45144.0,45744.000000,45744.000000,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,18.420805,70.217606,12.290452,17.601105,0.059136,0.059133,0.000002,0.0,20.273233,20.239752,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,5.448446,17.550882,4.184081,6.489564,0.383145,0.383141,0.000327,0.0,7.129466,5.843220,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2.409000,8.013013,-4.591000,-0.921747,0.000000,0.000000,0.000000,0.0,3.709000,7.559000,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,14.459001,57.654792,9.659000,12.704643,0.000000,0.000000,0.000000,0.0,14.709001,15.059000,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,17.909000,72.093994,12.659000,17.134960,0.000000,0.000000,0.000000,0.0,19.009001,18.959000,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,22.159000,85.082632,15.359000,22.388818,0.000000,0.000000,0.000000,0.0,25.009001,25.109001,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,39.359001,100.000000,22.809000,40.310566,20.000000,20.000000,0.070000,0.0,44.709000,33.508999,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(2).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(3).ValuesAsNumpy()
daily_rain_sum = daily.Variables(4).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(5).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(6).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(7).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(8).ValuesAsNumpy()
daily_sunrise = daily.Variables(9).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(10).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(11).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(12).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(13).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(14).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(15).ValuesAsNumpy()
daily_sunset = daily.Variables(16).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(17).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(18).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(19).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}

daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["rain_sum"] = daily_rain_sum
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["sunrise"] = daily_sunrise
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["sunset"] = daily_sunset
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe.to_csv("daily_weather_data.csv.zip")
daily_dataframe.describe()

,weather_code,temperature_2m_mean,apparent_temperature_mean,daylight_duration,rain_sum,wind_speed_10m_max,shortwave_radiation_sum,temperature_2m_max,apparent_temperature_max,sunrise,sunshine_duration,snowfall_sum,wind_gusts_10m_max,et0_fao_evapotranspiration,temperature_2m_min,apparent_temperature_min,sunset,precipitation_sum,precipitation_hours,wind_direction_10m_dominant
count,1906.000000,1906.000000,1906.000000,1906.000000,1906.000000,1906.000000,1906.000000,1906.000000,1906.000000,1906.0,1906.000000,1906.000000,1906.000000,1906.000000,1906.000000,1906.000000,1906.0,1906.000000,1906.00000,1906.000000
mean,18.096012,18.420805,17.601105,43661.214844,1.419203,20.672918,18.244574,22.187016,21.621981,0.0,36604.433594,0.000037,39.487614,3.704298,14.728859,13.846801,0.0,1.419255,2.17681,210.261597
std,24.709265,4.819868,5.941719,6285.461914,4.781114,6.713278,7.563122,5.276745,6.460413,0.0,9689.981445,0.001603,11.398125,1.915077,4.434433,5.768908,0.0,4.781110,4.62944,99.406265
min,0.000000,6.198583,1.732936,34619.460938,0.000000,5.991594,1.380000,8.909000,5.366825,0.0,0.000000,0.000000,14.400000,0.570944,2.409000,-0.921747,0.0,0.000000,0.00000,0.275439
25%,1.000000,14.554836,12.814921,37595.367188,0.000000,15.634115,11.372500,17.809000,16.306058,0.0,31562.853516,0.000000,31.680000,2.043561,11.309000,9.369694,0.0,0.000000,0.00000,102.042212
50%,3.000000,17.786082,17.053037,43430.173828,0.000000,19.870237,17.945001,21.309000,20.891127,0.0,37948.917969,0.000000,37.799999,3.346098,14.559000,13.574208,0.0,0.000000,0.00000,255.556435
75%,51.000000,22.009520,22.432214,49796.937500,0.200000,24.890655,25.585000,26.059000,26.786748,0.0,45450.707031,0.000000,45.989998,5.208264,18.159000,18.584207,0.0,0.200000,2.00000,288.104652
max,71.000000,33.619419,32.203854,52936.417969,68.300003,52.972626,30.290001,39.359001,40.310566,0.0,47925.933594,0.070000,115.919998,10.137957,28.009001,27.251232,0.0,68.300003,24.00000,359.999969
